## Ejercicio: búsqueda difusa

Dado el siguiente DataFrame, aplicar la búsqueda difusa de strings para la primera palabra de la descripción de cada producto (columna denominación)

Fuentes:
- http://inal.sifega.anmat.gov.ar/consultadeRegistroAlimentos/
- http://datos.salud.gob.ar/dataset/alimentos-libres-de-gluten/archivo/853d467c-5de5-46ae-bf04-5642c4960d1d

In [17]:
!pip install thefuzz[speedup]

In [18]:
import pandas as pd
from thefuzz import fuzz, process

In [19]:
url = "http://datos.salud.gob.ar/dataset/2ebd602e-3138-40c0-847a-f3eb8f520887/resource/853d467c-5de5-46ae-bf04-5642c4960d1d/download/alimentos-libres-de-gluten.csv"
df = pd.read_csv(url)

In [20]:
df.sample()

,marca,nombre_fantasia,denominacion,RNPA
3508,Villa del campo,No registra,Dulce de leche libre de gluten,02-520329


In [21]:
df['denominacion'].value_counts()

,count
denominacion,
Dulce de leche libre de gluten,74
Queso cremoso libre de gluten,63
Duraznos amarillos en mitades comunes en jarabe diluido – Libre de gluten,47
Queso sardo libre de gluten,45
Queso tybo libre de gluten,44
...,...
Producto de harina de maíz tipo fideo seco - Libre de gluten - (en sus diversos tamaños y formatos),1
Pasta fresca rellena sobre congelada: ravioles libre de gluten sin TACC,1
Pasta fresca rellena sobrecongelada: sorrentinos libre de gluten sin T.A.C.C.,1


Vemos que hay muchas palabras similares pero escritas distinto

- "libre de gluten" / "Libre de Gluten"  
- "sobre congelada" / "sobrecongeladas"  
- etc

In [22]:
# Contar nulos en "denominacion"
denominacion_nulo = df["denominacion"].isnull()

denominacion_nulo.sum()

1

In [23]:
# Ver los casos donde "denominacion" es nulo
df[denominacion_nulo]

,marca,nombre_fantasia,denominacion,RNPA
6578,;02-567232,NaN,NaN,NaN


In [24]:
# Descartar casos donde "denominacion" es nulo
df.dropna(subset="denominacion", inplace=True)
df[denominacion_nulo]

<ipython-input-24-3105187d3121>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[denominacion_nulo]


,marca,nombre_fantasia,denominacion,RNPA


Queremos contar la cantidad de alimentos por categoría ("Queso", "Yogur"...).

Para eso vamos a:
 1. Tomar la primer palabra de "denominacion"
 2. Ver con .value_counts() el resultado

Ahí veremos que hay palabras escritas con variaciones, de número u otras (ej "Lomitos", "Lomito"). Entonces vamos a intentar normalizar las palabras. Para eso tenemos dos enfoques:

A) Normalizar con pasos conocidos:
 1. Pasar a minúsculas
 2. Sacar espacios
 3. Sacar tildes
 4. Sacar puntuación

B) Normalizar palabras infrecuentes.
 1. Tomamos las palabras infrecuentes. Para eso podríamos tomar las que están por debajo de cierto percentil (ej. 1%), o las que aparecen una sola vez
 2. Luego, utilizando búsqueda difusa, intentamos matchear las palabras infrecuentes con las más frecuentes

In [25]:
def take_first_word(x):
  return x.split()[0]


df["simple"] = df["denominacion"].apply(take_first_word)
df["simple"].value_counts()

,count
simple,
Queso,1493
Leche,682
Alimento,668
Mermelada,549
Yogur,503
...,...
Estabilizante,1
Lomitos,1
Perejil,1


In [26]:
import unicodedata
import string

In [37]:
# A Normalizacion simple

def normalize_denominacion_simple(x):
  # a minuscula
  x = x.lower()
  # sin espacios
  x = x.strip()
  # sin acentos
  sin_acentos = unicodedata.normalize('NFKD', x)
  x = ''.join(c for c in sin_acentos if unicodedata.category(c) != 'Mn')
  # sin signos de puntuacion
  x = x.translate(str.maketrans('', '', string.punctuation + '–'))
  return x



In [28]:
df["simple"] = df["simple"].apply(normalize_denominacion_simple)

In [29]:
# B Normalizacion con palabras frecuentes
def get_match(string, lista_strings):
    """Para buscar coincidencias entre nombre con los nombres"""
    best_matches = process.extractBests(string.strip(), lista_strings, scorer=fuzz.ratio)
    mejor_match, puntaje = best_matches[0]
    return mejor_match, puntaje




In [30]:
# p = df["simple"].value_counts()
# un_solo_valor = p[p == 1].index
# filtro = df["simple"].isin(un_solo_valor)
# df_filtrado = df[filtro]
# lista_strings = df_filtrado["simple"].to_list()
# df_filtrado[["mejor_match", "puntaje"]] = df_filtrado["simple"].apply(lambda x: get_match(x, lista_strings)).apply(pd.Series)
# df_filtrado[df_filtrado["simple"] != df_filtrado["mejor_match"]]

In [31]:
lista_strings = df["simple"].to_list()
df["simple"] = df["simple"].apply(lambda x: get_match(x, lista_strings)[0])

In [32]:
df.head()

,marca,nombre_fantasia,denominacion,RNPA,simple
0,Abedul,No registra,"Aceite de maíz, libre de gluten, sin TACC",02-508542,aceite
1,Abedul,No registra,Aceite de maíz - Libre de gluten,02-519714,aceite
2,Abedul,No registra,Aceite de oliva virgen libre de gluten.,02-598508,aceite
3,Alimentos Tomy,No registra,Aceite de oliva extra libre de gluten,03003988-2,aceite
4,Almalegre,No registra,Aceite de oliva virgen extra – Libre de gluten,13043916,aceite


In [38]:
df["denominacion"] = df["denominacion"].apply(normalize_denominacion_simple)
df.head()

,marca,nombre_fantasia,denominacion,RNPA,simple
0,Abedul,No registra,aceite de maiz libre de gluten sin tacc,02-508542,aceite
1,Abedul,No registra,aceite de maiz libre de gluten,02-519714,aceite
2,Abedul,No registra,aceite de oliva virgen libre de gluten,02-598508,aceite
3,Alimentos Tomy,No registra,aceite de oliva extra libre de gluten,03003988-2,aceite
4,Almalegre,No registra,aceite de oliva virgen extra libre de gluten,13043916,aceite


In [42]:
regex = r'\s*(sin\s*TACC|libre\s*de\s*gluten|libre\s*de\s*gluten)\s*'

df["denominacion"] = df["denominacion"].str.replace(regex, "", case=False, regex=True)
df.rename(columns={
    "denominacion": "descripcion",
    "simple": "tipo_producto"
}, inplace=True)

df.head()

,marca,nombre_fantasia,descripcion,RNPA,tipo_producto
0,Abedul,No registra,aceite de maiz,02-508542,aceite
1,Abedul,No registra,aceite de maiz,02-519714,aceite
2,Abedul,No registra,aceite de oliva virgen,02-598508,aceite
3,Alimentos Tomy,No registra,aceite de oliva extra,03003988-2,aceite
4,Almalegre,No registra,aceite de oliva virgen extra,13043916,aceite
